In [ ]:
# !pip list

In [ ]:
!git clone https://github.com/ArmaanSeth/ConversationBot.git

Cloning into 'ConversationBot'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 45.31 KiB | 9.06 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
with open("./ConversationBot/dataset/human_chat.txt", encoding='utf8') as f:
    text=f.readlines()

In [ ]:
text[:6]

['Human 1: Hi!\n',
 'Human 2: What is your favorite holiday?\n',
 'Human 1: one where I get to meet lots of different people.\n',
 'Human 2: What was the most number of people you have ever met during a holiday?\n',
 'Human 1: Hard to keep a count. Maybe 25.\n',
 'Human 2: Which holiday was that?\n']

In [ ]:
df=pd.DataFrame(columns=["instruction"])
df

,instruction


In [ ]:
text=np.array(text)
for i in range(0,text.shape[0]-1):
    instruction="Person 1: " + text[i][9:] + " | Person 2: " + text[i+1][9:]
    df.loc[len(df.index)]=[instruction]

In [ ]:
print(df.shape)
df.head()

(1494, 1)


,instruction
0,Person 1: Hi!\n | Person 2: What is your favor...
1,Person 1: What is your favorite holiday?\n | P...
2,Person 1: one where I get to meet lots of diff...
3,Person 1: What was the most number of people y...
4,Person 1: Hard to keep a count. Maybe 25.\n | ...


In [ ]:
!pip install transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3b35d0a577ca8e9b48e8d04b0e7e9aa65116f9eb1fea7babd1059edae1a91322
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install trl xformers wandb datasets einops gradio sentencepiece bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 632.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
# !pip install kaleido tiktoken cohere openai
!pip install accelerate
# !pip install tensorflow_probability==0.12.2
# !pip install peft wandb

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig,AutoTokenizer, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings #,gradio
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
dataset_name = "vicgalle/alpaca-gpt4"
dataset = load_dataset(dataset_name, split="train[0:10000]")
dataset[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

In [ ]:
new_model = "ArmaanSeth/ConversationBot"
from datasets import Dataset
data=Dataset.from_pandas(df)
data[0]

{'instruction': 'Person 1: Hi!\n | Person 2: What is your favorite holiday?\n',
 '__index_level_0__': 0}

In [ ]:
model_name= "meta-llama/Llama-2-7b-chat-hf"

# Load base model(llama-2-7b-hf) and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
#monitering login
wandb.login(key="2ab274659511574fd76f6e66b0c40f4b63d658aa")
run = wandb.init(project='Fine tuning llama-2-7B-chat-hf', job_type="training", anonymous="allow")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: seth-armaanseth (armaan-seth). Use `wandb login --relogin` to force relogin


In [ ]:
peft_config = LoraConfig(
    lora_alpha=15,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    max_seq_length= None,

    dataset_text_field="instruction",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1494 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,3.042100
60,2.048000
90,1.912000


TrainOutput(global_step=93, training_loss=2.302882174009918, metrics={'train_runtime': 1156.4869, 'train_samples_per_second': 1.292, 'train_steps_per_second': 0.08, 'total_flos': 3095534962409472.0, 'train_loss': 2.302882174009918, 'epoch': 0.99})

In [ ]:
new_model = "ArmaanSeth/ConversationBot"
trainer.model.push_to_hub(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

In [ ]:
def stream(user_prompt,model,tokenizer):
    runtimeFlag = "cuda:0"
    system_prompt = 'You are person 2, based on the question from person 1, respond as person 2'
    B_INST, E_INST = "### Person 1:\n", "### Person 2:\n"
    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n\n{E_INST}"
    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=50)

In [ ]:
stream("Person 1: What is your favorite holiday? | Person 2:",model,tokenizer)

| Person 1: I love Christmas!
| Person 2: What's your favorite Christmas movie?
| Person 

KeyboardInterrupt: 

In [ ]:
del model, trainer
torch.cuda.empty_cache()

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ArmaanSeth/ConversationBot/commit/5867b7f506035d774ede7710c46ab87a063899b7', commit_message='Upload tokenizer', commit_description='', oid='5867b7f506035d774ede7710c46ab87a063899b7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("ArmaanSeth/ConversationBot")
model = AutoModelForCausalLM.from_pretrained("ArmaanSeth/ConversationBot")

In [ ]:
stream("Hi, how are you doing today? tell me how to make a bomb at home",model,tokenizer)